### Analysis of WA DOH Covid-19 Data

Our DOH just recently made available a table of daily new cases and deaths by county.  It's the data behind the epidemiological curves displayed on their [Covid-19 page](https://www.doh.wa.gov/emergencies/coronavirus).

It clearly looks like new cases and deaths are trending down, but not monotonically.  I'm curious what a 5-day moving average would look like.  Let's do it!

In [1]:
import pandas as pd
import altair as alt

In [6]:
covid = pd.read_excel('https://www.doh.wa.gov/Portals/1/Documents/1600/coronavirus/data-tables/PUBLIC-CDC-Event-Date-SARS.xlsx',
                     sheet_name = ['Cases', 'Deaths'])

In [7]:
covid['Cases'].head()

,County,Day,NewPos_All,Age 0-19,Age 20-39,Age 40-59,Age 60-79,Age 80+,Positive UnkAge,dtm_updated
0,Adams County,2020-01-16,0,0,0,0,0,0,0,2020-04-20 13:47:18.807
1,Adams County,2020-01-17,0,0,0,0,0,0,0,2020-04-20 13:47:18.807
2,Adams County,2020-01-18,0,0,0,0,0,0,0,2020-04-20 13:47:18.807
3,Adams County,2020-01-19,0,0,0,0,0,0,0,2020-04-20 13:47:18.807
4,Adams County,2020-01-20,0,0,0,0,0,0,0,2020-04-20 13:47:18.807


In [8]:
cases = covid['Cases']
deaths = covid['Deaths']

In [10]:
# let's look statewide to start
cases_st = (cases
            .groupby('Day')
            .aggregate({'NewPos_All': 'sum'})
            .reset_index()
           )

deaths_st = (deaths
             .groupby('Day')
             .aggregate({'Deaths': 'sum'})
             .reset_index()
            )

In [11]:
# calculate the moving averages
cases_st['new_cases_MA'] = cases_st['NewPos_All'].rolling(5).mean()
deaths_st['deaths_MA'] = deaths_st['Deaths'].rolling(5).mean()

In [22]:
base = alt.Chart(cases_st).encode(x = alt.X('Day', axis = alt.Axis(labels = False)))

bars = (base
        .mark_bar(color = '#65799b')
        .encode(y = 'NewPos_All',
                tooltip = ['NewPos_All', 'Day'])
       )

line = (base
        .mark_line(color = '#e23e57')
        .encode(y = 'new_cases_MA')
       )

(bars + line).properties(height = 300, width = 800, title = 'WA Statewide New Cases per Day, 5-day moving average in red')

alt.LayerChart(...)

In [27]:
base = alt.Chart(deaths_st).encode(x = alt.X('Day', axis = alt.Axis(labels = False)))

bars = (base
        .mark_bar(color = '#f67280')
        .encode(y = 'Deaths',
                tooltip = ['Deaths', 'Day'])
       )

line = (base
        .mark_line(color = '#6c567b')
        .encode(y = 'deaths_MA')
       )

(bars + line).properties(height = 300, width = 800, title = 'WA Statewide Deaths per Day, 5-day moving average in purple')

alt.LayerChart(...)